In [1]:
import pandas as pd
import numpy as np

#display
pd.set_option('display.max_colwidth', None)

df_case = pd.read_pickle('df_cases_200906.gzip')
df_lable = pd.read_pickle('df_label_200906.gzip')

````
vectorized code runs faster than looping
```

In [2]:

%%time
#Create a column to determine validity of each row 
df_case['Is_Valid'] = np.where((df_case['IsExecuted'] == False) | (df_case['QualityScore'] < 0.81), False,True)
df_case = df_case.drop(columns=['Language','StartDate','DocumentType','IsExecuted','QualityScore'])

#Inserting columns at position 2 and 3
df_case.insert(2,'InvalidFileNames','')
df_case.insert(3,'ValidFileNames','')

#Fill Name in column InvalidFileNames and ValidFileNames to groupby later
df_case['InvalidFileNames'] = np.where((df_case['Is_Valid'] == False),df_case['FileName'],None)
df_case['ValidFileNames'] = np.where((df_case['Is_Valid'] == True),df_case['FileName'],None)
df_case['OcrText'] = np.where((df_case['Is_Valid'] == True),df_case['OcrText'],None)

df_case = df_case.drop(columns=['FileName','Is_Valid'])

df_case = df_case.groupby('CaseId').agg({'InvalidFileNames': lambda x: list(filter(None, x)),
                               'ValidFileNames': lambda x: list(filter(None, x)),
                               'OcrText': lambda x: ' '.join(list(filter(None, x)))
                              }).reset_index()

df_final = pd.merge(df_case, df_lable, on='CaseId')

CPU times: user 54.9 ms, sys: 4.17 ms, total: 59.1 ms
Wall time: 61.3 ms


In [3]:
df_final.tail(5)

,CaseId,InvalidFileNames,ValidFileNames,OcrText,label_1,label_2
1093,003061230659,[003061230659_72651667_Order form_978-0-661-06636-2.pdf],[],,True,False
1094,003061230710,"[003061230710_80047544_other documents_978-1-02-279791-8.pdf, 003061230710_59256366_Order form_978-1-209-37083-5.pdf]",[],,True,False
1095,003061230728,"[003061230728_79408066_Master contract_978-0-14-891566-4.pdf, 003061230728_56717174_Amendments_978-1-4471-4999-6.pdf]",[003061230728_74076581_Amendments_978-0-14-763189-3.pdf],None attorney spend tend miss appear.,True,False
1096,003061230748,[003061230748_65193716_Contract Documents_978-0-9561206-8-7.pdf],[],,True,False
1097,003061230757,[003061230757_84690982_other documents_978-0-10-551297-4.pdf],[003061230757_72990476_Contract Documents_978-1-4943-5720-7.pdf],Determine go network.,False,False


In [4]:
df_final.to_pickle("./df_final.gzip")

In [5]:
#df = pd.read_pickle('df_final.gzip')